## LLM zoomcamp Homework 1

### Q 1 

Run Elastic Search 8.17.6, and get the cluster information. What's the version.build_hash value?


```bash
docker stop $(docker ps -q)
docker rm $(docker ps -aq)

docker pull docker.elastic.co/elasticsearch/elasticsearch:8.17.6

docker run -p 9200:9200 -e "discovery.type=single-node" -e "ES_JAVA_OPTS=-Xms1g -Xmx1g" --memory="2g" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.17.6

In [1]:
%pip install 'elasticsearch>=8.0.0,<9.0.0'


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from elasticsearch import Elasticsearch
import os 
from dotenv import load_dotenv
import requests

In [3]:
response = requests.get("http://localhost:9200")

In [4]:
data = response.json()

In [5]:
print(f"version.build_hash value is {data["version"]["build_hash"]} ")

version.build_hash value is dbcbbbd0bc4924cfeb28929dc05d82d662c527b7 


In [6]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q 2

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Which function do you use for adding your data to elastic?

In [7]:
import minsearch

/workspaces/llm-zoomcamp/week-1/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [8]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
index_name = "llm-zoomcamp"  # Use a valid, lowercase name

mapping = {
    "mappings": {
        "properties": {
            "course": {"type": "keyword"},
            "question": {"type": "text"},
            "text": {"type": "text"},
            "section": {"type": "text"}
        }
    }
}

try:
    es_client.indices.create(index=index_name, body=mapping)
except Exception as e:
    print(e)

# Delete the index if it already exists (optional, for a clean start)
#if es_client.indices.exists(index=index_name):
#    es_client.indices.delete(index=index_name)

# Create the index with the mapping
#es_client.indices.create(index=index_name, body=mapping)

BadRequestError(400, 'resource_already_exists_exception', 'index [llm-zoomcamp/bEAG42oDRaySHIc-482_Iw] already exists')


In [9]:
from tqdm.auto import tqdm

/workspaces/llm-zoomcamp/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:02<00:00, 418.68it/s]


In [11]:
es_client = Elasticsearch('http://localhost:9200') 

In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:02<00:00, 447.39it/s]


In [13]:
print(f"We use es_client.index to add data to elastic.")

We use es_client.index to add data to elastic.


### Q3. Searching
Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

In [14]:
query_0 = "How do execute a command on a Kubernetes pod?"

query = {
    "query": {
        "multi_match": {
            "query": query_0,
            "fields": ["question^4", "text"],
            "type": "best_fields"
        }
    }
}

response = es_client.search(index=index_name, body=query)
top_hits = response['hits']['hits'][0]["_score"]

print(f"Top hits score for query '{query_0}': {top_hits}")


Top hits score for query 'How do execute a command on a Kubernetes pod?': 44.58877


### Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

In [37]:
query_0 = "How do copy a file to a Docker container?"

query = {
    "query": {
        "bool": {
            "must": [ 
                {
            "multi_match": {
                "query": query_0,
                "fields": ["question^4", "text"],
                "type": "best_fields"
        }
    }
    ], 
    "filter": [ { "term": {
                "course": "machine-learning-zoomcamp"
            }
        }
    ]
    }
} 
}

response = es_client.search(index=index_name, body=query)
unique_questions = []
for hit in response["hits"]["hits"]:
    print( hit["_source"]["question"])
    if hit["_source"]["question"] not in unique_questions:
        unique_questions.append(hit["_source"]["question"])
    if len(unique_questions) >= 3:
        break   

print(f"3rd question: {unique_questions[2]}")

How do I debug a docker container?
How do I debug a docker container?
How do I debug a docker container?
How do I debug a docker container?
How do I debug a docker container?
How do I debug a docker container?
How do I debug a docker container?
How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from my local machine to docker container?


IndexError: list index out of range

### Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

context_template = """
Q: {question}
A: {text}
""".strip()


In [16]:
context_template = """
Q: {question}
A: {text}
""".strip()

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = "\n\n".join([
        context_template.format(
            question= doc['_source']['question'],
            text= doc['_source']['text']
        ) for doc in search_results
        ] )
    
    #context = ""
    
    #for doc in search_results:
        
    #    context = context + f"section: {doc['_source']['section']}\nquestion: {doc['_source']['question']}\nanswer: {doc['_source']['text']}\n\n"
    
    #prompt = context_template.format(question=query, context=context).strip()
    
    prompt = prompt_template.format(question = query, context=context)

    return prompt

In [17]:
response["hits"]["hits"]

[{'_index': 'llm-zoomcamp',
  '_id': '2mdicpcBwLXuqoQJqubq',
  '_score': 22.97349,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I copy files from my local machine to docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'llm-zoomcamp',
  '_id': 'JmdicpcBwLXuqoQJneMT',
  '_score': 22.97349,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command

In [18]:
prompt = build_prompt(query_0, response["hits"]["hits"])

In [19]:
print(f"The lenght of the prompt is { len(prompt)}")

The lenght of the prompt is 1538


### Q6. Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

Let's calculate the number of tokens in our query:

encoding = tiktoken.encoding_for_model("gpt

In [20]:
!pip install tiktoken


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [21]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [22]:
tokens = encoding.encode(prompt)
num_tokees = len(tokens)
print(f"The number of tokens in the prompt is {num_tokees}")

The number of tokens in the prompt is 335


In [23]:
decoded_prompt = encoding.decode(tokens)
print(decoded_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT: 
Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as fol

### Q 7 Bonus: generating the answer (ungraded)
Let's send the prompt to OpenAI. What's the response?

Note: you can replace OpenAI with Ollama. See module 2.

In [24]:
import openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."

In [26]:
client = openai.OpenAI()  # This uses the OPENAI_API_KEY environment variable

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.0,
    max_tokens=1000
)

answer = response.choices[0].message.content
print(answer)
print(f"Number of tokens in the answer: {len(encoding.encode(answer))}")

To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is:

```
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```

This command allows you to copy a file or directory from your local machine into a running Docker container.
Number of tokens in the answer: 64


### Bonus: calculating the costs (ungraded)
Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices here

On June 17, the prices for gpt4o are:

Input: $0.005 / 1K tokens
Output: $0.015 / 1K tokens
You can redo the calculations with the values you got in Q6 and Q7.

Total cose is Here’s how you calculate the cost:

**Given:**
- 150 input tokens/request
- 250 output tokens/request
- 1000 requests
- Input price: $0.005 per 1K tokens
- Output price: $0.015 per 1K tokens

---

**Final answer:**  
> It will cost **$4.50** to run 1000 requests with these parameters.

Total cose is Here’s how you calculate the cost:

**Given:**
- 335 input tokens/request
- 64 output tokens/request
- 1000 requests
- Input price: $0.005 per 1K tokens
- Output price: $0.015 per 1K tokens

---

**Final answer:**  
> It will cost **$2.635** to run 1000 requests with these parameters.